In [2]:
import os
import zipfile
import pandas as pd
import basedosdados as bd

INPUT = os.path.join(os.getcwd(), "input")
OUTPUT = os.path.join(os.getcwd(), "output")

os.makedirs(INPUT, exist_ok=True)
os.makedirs(OUTPUT, exist_ok=True)

In [39]:
def read_sheet(sheet_name: str, skiprows: int = 7) -> pd.DataFrame:
    return pd.read_excel(
        os.path.join(
            INPUT,
            "mat_ed_especial_aee_uf_rede_2019_2022.xlsx"
        ),
        skiprows=skiprows,
        sheet_name=sheet_name
    )

In [40]:
excel_data = pd.ExcelFile(os.path.join(
            INPUT,
            "mat_ed_especial_aee_uf_rede_2019_2022.xlsx"
        ))

# Get the sheet names
sheet_names = excel_data.sheet_names

In [41]:
dfs = {
    sheet_name: read_sheet(sheet_name)
    for sheet_name in sheet_names
}

In [42]:
dfs

{'tabmat':                                       NU_ANO_CENSO     NO_REGIAO  CO_REGIAO  \
 0                                             2022        Brasil        NaN   
 1                                             2022        Brasil        NaN   
 2                                             2022        Brasil        NaN   
 3                                             2022         Norte        1.0   
 4                                             2022         Norte        1.0   
 ..                                             ...           ...        ...   
 333                                           2019  Centro-Oeste        5.0   
 334                                           2019  Centro-Oeste        5.0   
 335                                           2019  Centro-Oeste        5.0   
 336                                            NaN           NaN        NaN   
 337  Fonte: Censo Escolar da Educação Básica/Inep.           NaN        NaN   
 
                 NO_UF SG_UF

In [43]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print()  # Adds a blank line for readability

Sheet: tabmat
Index(['NU_ANO_CENSO', 'NO_REGIAO', 'CO_REGIAO', 'NO_UF', 'SG_UF', 'CO_UF',
       'rede', 'MatEsp', 'MatEspAee'],
      dtype='object')



In [44]:
RENAME_COLUMNS = {
    'NU_ANO_CENSO':'ano',
    'SG_UF':'sigla_uf',  
    'MatEsp':'quantidade_matricula', 
    'MatEspAee':'quantidade_matricula_aee'}

In [45]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols_drop = [
        col
        for col in df.columns
        if col.startswith("NO_") 
        or col.startswith("CO") 
    ]

    return df.drop(columns=cols_drop)

dfs = {
    name: drop_unused_columns(
        df.rename(columns=RENAME_COLUMNS, errors="raise")
    )
    for name, df in dfs.items()
}

In [46]:
dfs

{'tabmat':                                                ano sigla_uf     rede  \
 0                                             2022      NaN      NaN   
 1                                             2022      NaN  Pública   
 2                                             2022      NaN  Privada   
 3                                             2022       RO      NaN   
 4                                             2022       RO  Pública   
 ..                                             ...      ...      ...   
 333                                           2019       DF      NaN   
 334                                           2019       DF  Pública   
 335                                           2019       DF  Privada   
 336                                            NaN      NaN      NaN   
 337  Fonte: Censo Escolar da Educação Básica/Inep.      NaN      NaN   
 
      quantidade_matricula  quantidade_matricula_aee  
 0               1527794.0                  568200.0  
 1

In [47]:
for sheet_name, df in dfs.items():
    print(f"Sheet: {sheet_name}")
    print(df.columns)  # This will print the column names of each DataFrame
    print() 

Sheet: tabmat
Index(['ano', 'sigla_uf', 'rede', 'quantidade_matricula',
       'quantidade_matricula_aee'],
      dtype='object')



In [48]:
dataframe = pd.DataFrame(dfs['tabmat']).dropna(subset=['sigla_uf', 'rede']).loc[lambda df: (df['sigla_uf'] != " ") & (df['rede'] != " ")]

In [49]:
dataframe[['quantidade_matricula','quantidade_matricula_aee']] = dataframe[['quantidade_matricula','quantidade_matricula_aee']].astype(int)

In [50]:
dataframe = dataframe.sort_values(by=['ano', 'sigla_uf'], ascending=[True, True])

In [51]:
dataframe

,ano,sigla_uf,rede,quantidade_matricula,quantidade_matricula_aee
259,2019,AC,Pública,10782,6395
260,2019,AC,Privada,325,261
295,2019,AL,Pública,21838,10145
296,2019,AL,Privada,1314,1832
262,2019,AM,Pública,17216,5074
...,...,...,...,...,...
47,2022,SE,Privada,1604,227
61,2022,SP,Pública,191708,68563
62,2022,SP,Privada,52146,942
22,2022,TO,Pública,15977,7222


In [52]:
path = os.path.join(
        OUTPUT, "educacao_especial_aee_matricula"
    )

os.makedirs(path, exist_ok=True)
dataframe.astype(str).to_csv(os.path.join(path, "data.csv"), index=False)